<a href="https://colab.research.google.com/github/Bjorn455215/B103021055/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyngrok flask_ngrok # for colab
!pip install -q flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 10.1 MB/s eta 0:00:00


In [12]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTOTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage,
    ConfirmTemplate,
    MessageAction,
    CarouselTemplate,
    CarouselColumn,
    URIAction,#第一個按鈕
    PostbackAction#第二個按鈕
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('Line_channel_accesstoken'))
handler = WebhookHandler(userdata.get('Line_channel_token'))

'''
除做伺服器外，也可以額外新增功能
@app.route("/callback", methods=['GET'])
def index():
  return'hello!'
'''

@app.route("/callback", methods=['POST'])#bot server自己在使用的伺服器，專門為了line bot使用
# 將 /callback 路徑與 callback 函式綁定，用於處理來自 LINE 平台的 POST 請求。這是 LINE Bot 的 Webhook URL。
def callback(): #用於驗證請求簽章，並將請求交給 LINE Bot SDK 處理。
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)#連接下面的def
    except InvalidSignatureError:
        abort(400)
    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)#啟動linebot要做的事情
##用於處理使用者傳送的文字訊息。
def handle_message(event):#用於根據使用者傳送的文字訊息做出回應。
    with ApiClient(configuration) as api_client: #ApiClient 是用於與 LINE API 進行通訊的客戶端物件。
        line_bot_api = MessagingApi(api_client)#MessagingApi 提供了各種與 LINE 訊息相關的功能，例如傳送訊息、回覆訊息等。
        action = event.message.text
        if action == 'confirm':#輸入 confirm 會顯示一個確認視窗，詢問使用者是否喜歡看青春之箱。

             #建立一個 ConfirmTemplate 物件，用於顯示確認視窗，詢問使用者是否喜歡看青春之箱。
             reply = TemplateMessage(
                 alt_text="這是確認視窗",
                 template=ConfirmTemplate(
                     text="你喜歡看青春之箱嗎？",
                     actions=[
                        MessageAction(label="是", text="我超愛"),
                        MessageAction(label="否", text="其實我非常喜歡 但我要說不")
                         ]
                   ))
             #建立一個 TemplateMessage 物件，將 ConfirmTemplate 作為其模板，並設定替代文字為 "這是確認視窗"。

        elif action == 'carousel':
          carousel_template= CarouselTemplate(
            columns=[
                CarouselColumn(
                    thumbnail_image_url='https://images.chinatimes.com/newsphoto/2020-01-09/1024/BBC400_P_03_02.jpg',
                    title='夏日大作戰',
                    text='細田守導演的日本科幻暨浪漫電影。',
                    actions=[
                        URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E5%A4%8F%E6%97%A5%E5%A4%A7%E4%BD%9C%E6%88%B0'),#按鈕1
                        MessageAction(label='投票', text='我投夏日大作戰一票')#按鈕2
                    ]
                ),
                CarouselColumn(
                    thumbnail_image_url='https://i.imgur.com/M9wqbiT.jpeg',
                    title='青春之箱',
                    text='日本漫畫家三浦糀創作的少年漫畫。',
                    actions=[
                        URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E9%9D%92%E6%98%A5%E4%B9%8B%E7%AE%B1'),#按鈕1
                        MessageAction(label='投票', text='我投青春之箱一票')#按鈕2
                    ]
                )
            ]
        )
          reply = TemplateMessage(
                 alt_text="這是輪播視窗",
                 template=carousel_template)

        else:
          reply = TextMessage(text = 'Thanks !')#輸入其他回復，則回覆Thanks !
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,#reply_token: 回覆 Token，用於識別要回覆的訊息。
                messages=[
                    reply#reply 物件是之前建立的 TemplateMessage物件，代表要回覆的訊息內容。
                ]
            )
        )

ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://d159-34-138-200-213.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:09:22] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:09:23] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:09:34] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:09:48] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:09:57] "POST /callback HTTP/1.1" 200 -
